### Hash Function

In [1]:
import hashlib
def hash_sha256(raw):
    raw = str(raw).encode('utf-8')
    return hashlib.sha256(raw).hexdigest() 

In [2]:
hash_sha256('torpe_blockchain')

'e5367197a1f12480ec761306f2fa9d15b494d5a80e5a806713df9e60943f4faf'

In [3]:
hash_sha256('torpe_blockchain')

'e5367197a1f12480ec761306f2fa9d15b494d5a80e5a806713df9e60943f4faf'

In [4]:
hash_sha256('monkey_brain')

'35ba7e0c2afd3e1175399e99c7e307ea78e62c9dd6e3cb39afc9604043a55e4b'

SHA-256 collisions have not been found yet. 

### Nonce

> Number that can only be used once

An arbitrary numer used in cryptography to ensure uniqueness and prevent the rerunning of transactions (known as replay attack). 

## Block

Blocks hold batches of valid transactions that are hashed and encoded into a Merkle tree Each block includes the cryptographic hash of the prior block in the blockchain, linking the two. The linked blocks form a chain. This iterative process confirms the integrity of the previous block, all the way back to the original genesis block.

### Genesis Block

A genesis block or block0 is the first block of a block chain. The genesis block is almost always hardcoded into the software of the applications that utilize its block chain. It is a special case in that it does not reference a previous block

In [5]:
import datetime

In [6]:
# Lets assume 5 person were given 100 coins each
state = {
    'Person_1': 100, 
    'Person_2': 100, 
    'Person_3': 100, 
    'Person_4': 100, 
    'Person_5': 100
}

In [7]:
block0_data = {
    'timestamp': datetime.datetime.now(),
    'index': 0,
    'previous': None,
    'transactions': [state] 
}

block0 = {
    'hash': hash_sha256(block0_data),
    'data': block0_data,
}

In [8]:
block0

{'data': {'index': 0,
  'previous': None,
  'timestamp': datetime.datetime(2019, 1, 24, 22, 2, 25, 304057),
  'transactions': [{'Person_1': 100,
    'Person_2': 100,
    'Person_3': 100,
    'Person_4': 100,
    'Person_5': 100}]},
 'hash': 'f6a260ede08bae762805d6754d8b832e460bbe76885107eff43321612d011689'}

This is the genesis block or block 0 here.

### Transactions

In [9]:
import random
def random_transaction(state):
    temp_list = list(state.keys())
    random.shuffle(temp_list)
    # randomly select two persons
    first_person = temp_list.pop()
    second_person = temp_list.pop()
    receive = random.randint(1, 10)
    give = -receive
    return {
        first_person:receive, 
        second_person:give
    }

In [10]:
test_transactions = [random_transaction(state) for x in range(5)]

In [11]:
test_transactions

[{'Person_2': 9, 'Person_5': -9},
 {'Person_1': 7, 'Person_2': -7},
 {'Person_1': -6, 'Person_2': 6},
 {'Person_2': 2, 'Person_3': -2},
 {'Person_4': -1, 'Person_5': 1}]

### Updating State

In [12]:
def update_state(transaction, state):
    state = state.copy()
    for key in transaction:
        state[key] = state.get(key) + transaction[key]
    return state

In [13]:
for transaction in test_transactions:
    state = update_state(transaction, state)

In [14]:
state

{'Person_1': 101,
 'Person_2': 110,
 'Person_3': 98,
 'Person_4': 99,
 'Person_5': 92}

### Valid Transactions

In [15]:
def check_transaction_validity(transaction, state):
    # check neg vs pos
    if sum(transaction.values()) is not 0:
        return False
    # check if amount in wallet to give
    for key in transaction.keys():
        if state.get(key) + transaction[key] < 0:
            return False
    return True

In [16]:
for transaction in test_transactions:
    print (check_transaction_validity(transaction, state))

True
True
True
True
True


In [17]:
# No balance
print (check_transaction_validity({'A': 5, 'B': -5}, {'A': 0, 'B': 0}))

False


In [18]:
# Bad transaction
print (check_transaction_validity({'A': 5, 'B': 5}, {'A': 50, 'B': 50}))

False


### Initial State

In [19]:
# Let us reset
# Lets assume 5 person were given 100 coins each
state = {
    'Person_1': 100, 
    'Person_2': 100, 
    'Person_3': 100, 
    'Person_4': 100, 
    'Person_5': 100
}

blockchain = []

In [20]:
# Adding the genesis block
blockchain.append(block0)

In [21]:
blockchain

[{'data': {'index': 0,
   'previous': None,
   'timestamp': datetime.datetime(2019, 1, 24, 22, 2, 25, 304057),
   'transactions': [{'Person_1': 100,
     'Person_2': 100,
     'Person_3': 100,
     'Person_4': 100,
     'Person_5': 100}]},
  'hash': 'f6a260ede08bae762805d6754d8b832e460bbe76885107eff43321612d011689'}]

### Non-genesis block / New block

In [22]:
def new_block(transactions, blockchain):
    previous_block = blockchain[-1]
    data = {
        'timestamp': datetime.datetime.now(),
        'index': previous_block['data']['index'] + 1,
        'previous': previous_block['hash'],
        'transactions': transactions,
        }

    block = {'hash': hash_sha256(data), 'data': data}
    return block

In [23]:
sample_transactions = [random_transaction(state) for x in range(50)]

In [24]:
sample_transactions

[{'Person_1': 9, 'Person_5': -9},
 {'Person_1': -10, 'Person_5': 10},
 {'Person_3': -10, 'Person_5': 10},
 {'Person_1': -4, 'Person_2': 4},
 {'Person_1': 2, 'Person_3': -2},
 {'Person_3': -9, 'Person_5': 9},
 {'Person_1': 3, 'Person_3': -3},
 {'Person_4': 1, 'Person_5': -1},
 {'Person_1': -10, 'Person_3': 10},
 {'Person_1': 7, 'Person_3': -7},
 {'Person_1': 2, 'Person_5': -2},
 {'Person_4': 4, 'Person_5': -4},
 {'Person_2': 7, 'Person_5': -7},
 {'Person_4': 5, 'Person_5': -5},
 {'Person_2': -5, 'Person_3': 5},
 {'Person_1': -5, 'Person_5': 5},
 {'Person_4': 7, 'Person_5': -7},
 {'Person_4': 4, 'Person_5': -4},
 {'Person_1': -1, 'Person_3': 1},
 {'Person_1': -2, 'Person_3': 2},
 {'Person_2': 2, 'Person_4': -2},
 {'Person_1': 1, 'Person_3': -1},
 {'Person_2': -9, 'Person_4': 9},
 {'Person_1': -7, 'Person_3': 7},
 {'Person_2': -3, 'Person_4': 3},
 {'Person_3': -9, 'Person_4': 9},
 {'Person_1': 3, 'Person_2': -3},
 {'Person_3': -6, 'Person_5': 6},
 {'Person_1': 10, 'Person_2': -10},
 {'Per

### Transactions per block

Bitcoin blocks used to contain fewer than 200 transactions and the largest number of transactions in a block was 1,976 at the time this answer was originally written (May 2013). In meanwhile (November 2017) the average number of transaction per block is well above 1500 with peaks above 2200.

In [25]:
# Although block size is 
transactions_per_block = 5
transaction_block = []

for transaction in sample_transactions:
    if check_transaction_validity(transaction, state):
        state = update_state(transaction, state)
        transaction_block.append(transaction)
        
        if len(transaction_block) >= transactions_per_block:
            blockchain.append(new_block(transaction_block, blockchain))
            transaction_block = []

In [26]:
import pprint
pp = pprint.PrettyPrinter()
for block in blockchain:
    pp.pprint(block)
    print('\n************************************************************************************\n')

{'data': {'index': 0,
          'previous': None,
          'timestamp': datetime.datetime(2019, 1, 24, 22, 2, 25, 304057),
          'transactions': [{'Person_1': 100,
                            'Person_2': 100,
                            'Person_3': 100,
                            'Person_4': 100,
                            'Person_5': 100}]},
 'hash': 'f6a260ede08bae762805d6754d8b832e460bbe76885107eff43321612d011689'}

************************************************************************************

{'data': {'index': 1,
          'previous': 'f6a260ede08bae762805d6754d8b832e460bbe76885107eff43321612d011689',
          'timestamp': datetime.datetime(2019, 1, 24, 22, 2, 25, 581327),
          'transactions': [{'Person_1': 9, 'Person_5': -9},
                           {'Person_1': -10, 'Person_5': 10},
                           {'Person_3': -10, 'Person_5': 10},
                           {'Person_1': -4, 'Person_2': 4},
                           {'Person_1': 2, 'Person_3':